In [ ]:
import json
import torch
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
!pip install datasets
from datasets import Dataset
import time
import random
import numpy as np

In [ ]:
#Debemos fijar el parámetro de random seed siempre al mismo valor y después cargar el modelo para que el modelo devuelva siempre el mismo valor
# SET RANDOM SEED
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


#ENTRENAMIENTO

In [ ]:
OPTION_LETTERS = ["A", "B", "C", "D", "E"]

# Cargar el dataset de entrenamiento o test
def load_dataset(file_path):
  with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)["data"]

  # Convertir el JSON a un formato adecuado
  formatted_data = []
  max_labels = 0
  for item in data:
      context = item["context"]
      question = item["question"]
      choices = [choice["text"] for choice in item["choices"]]
      correct_answer = next(i for i, choice in enumerate(item["choices"]) if choice["type"] == "correct answer")  #El número!!
      prompt = f"Dado el siguiente contexto:\n{context}\nPregunta: {question}\nOpciones:\n" + \
                "\n".join([f"{OPTION_LETTERS[i]}. {opt}" for i, opt in enumerate(choices)]) + \
                "\nSelecciona la respuesta correcta:"
      formatted_data.append({"text": prompt, "label": correct_answer})
      max_labels = max(max_labels, len(choices))
  return formatted_data, max_labels

In [ ]:
# Cargar datos
train_data, max_labels_train = load_dataset("1-training.json")
dev_data, max_labels_dev = load_dataset("1-dev.json")
num_labels = max_labels_train

# Cargar tokenizer y modelo
model_name = "datificate/gpt2-small-spanish"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 no tiene pad token
model = GPT2ForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
model.config.pad_token_id = model.config.eos_token_id


In [ ]:
class ClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = tokenizer(item["text"], padding="max_length", truncation=True,
                             max_length=self.max_length, return_tensors="pt")
        return {
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": torch.tensor(item["label"])
        }

train_dataset = ClassificationDataset(train_data, tokenizer)
dev_dataset = ClassificationDataset(dev_data, tokenizer)


In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
import time
import os
os.environ["WANDB_DISABLED"] = "true"

start_time = time.time()

training_args = TrainingArguments(
    output_dir="./gpt2-results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,

    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    weight_decay=0.1,
    logging_dir="./gpt2-logs",
    logging_steps=10,
    load_best_model_at_end=True,
    fp16=True,
    report_to="none"
)



trainer_gpt2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
)


trainer_gpt2.train()

end_time = time.time()
elapsed_time = end_time - start_time

# Mostrar tiempo en minutos y segundos
mins, secs = divmod(elapsed_time, 60)
print(f"Tiempo total de entrenamiento: {int(mins)} min {int(secs)} sec")


trainer_gpt2.save_model("modelo_finetuned")
tokenizer.save_pretrained("modelo_finetuned")

#EVALUACIÓN

In [ ]:
modelo_finetuned = "modelo_finetuned"
tokenizer_gpt2 = GPT2Tokenizer.from_pretrained(modelo_finetuned)
modelGPT2 = GPT2ForSequenceClassification.from_pretrained(modelo_finetuned)

with open("4-test.json", "r", encoding="utf-8") as f:
    test_dataset = json.load(f)["data"]

def format_prompt(context, question, choices):
  prompt = f"Dado el siguiente contexto:\n{context}\nPregunta: {question}\nOpciones:\n" + \
         "\n".join([f"{OPTION_LETTERS[i]}. {opt}" for i, opt in enumerate(choices)]) + \
         "\nRespuesta correcta:"
  return prompt

import torch.nn.functional as F
def predict_answerGPT2(inputs, choices, num_choices, temperature = 1.0):
  with torch.no_grad():
    outputs = modelGPT2(**inputs)

    logits = outputs.logits.squeeze(0)[:num_choices]  #quitamos opción extra, si la hay
    sacled_logits = logits / temperature
    probs = F.softmax(sacled_logits, dim=-1)

    predicted_index = torch.argmax(probs).item()
    if predicted_index >= num_choices:
      print(f"Error: Índice de predicción {predicted_index} fuera de rango para {num_choices} opciones. Saltamos la pregunta.")
      return None

    predicted_answer = choices[predicted_index]
  return predicted_answer


In [ ]:
correct_predictionsGPT2 = 0
total_questions = len(test_dataset)
predictionsGPT2 = []
# print(f"Total de preguntas: {total_questions}")

start_time = time.time()
for item in test_dataset:
  is_correct_prediction = False
  context = item["context"]
  question = item["question"]
  choices = [choice["text"] for choice in item["choices"]]
  correct_answer = next(choice["text"] for choice in item["choices"] if choice["type"] == "correct answer")

  num_choices = len(choices)
  if num_choices > num_labels:
    print(f"Error: Pregunta con {num_choices} opciones. Supera el máximo permitido: {max_labels}")
    continue

  prompt = format_prompt(context, question, choices)
  inputsGPT2 = tokenizer_gpt2(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)

  predicted_answerGPT2 = predict_answerGPT2(inputsGPT2, choices, num_choices)


  if correct_answer == predicted_answerGPT2:
    correct_predictionsGPT2 += 1
    is_correct_prediction = True


  predictionsGPT2.append({
    "context": context,
    "question": question,
    "choices": choices,
    "correct_answer": correct_answer,
    "predicted_answer": predicted_answerGPT2,
    "is_correct": is_correct_prediction
  })


end_time = time.time()
elapsed_time = end_time - start_time
print(f"Tiempo de ejecución: {elapsed_time:.2f} segundos")
print()

with open("predictionsGPT2.json", "w", encoding="utf-8") as f:
    json.dump(predictionsGPT2, f, indent=4, ensure_ascii=False)


print(f"Total de preguntas evaluadas: {total_questions}")
print(f"Respuestas correctas GPT2: {correct_predictionsGPT2}")
print(f"Precisión del modelo GPT2: {correct_predictionsGPT2 / total_questions:.2%}")
print()
